# Vision Transformer (ViT)

ViT 模型是由 Google Brain 团队在 2021 ICLR上发表的 [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929) 中提出。这是第一篇成功在 ImageNet 上训练 Transformer 编码器的论文，与常见的卷积架构相比，取得了非常好的效果。

**论文摘要**：尽管 Transformer 架构已成为自然语言处理任务的默认标准，但其应用于计算机视觉的领域仍然有限。在视觉领域，注意力机制要么与卷积神经网络结合使用，要么用于替换卷积网络中的某些组件，同时保持其整体结构不变。我们表明，这种对 CNN 的依赖并非必要，直接应用于图像块序列的纯 Transformer 在图像分类任务上可以表现出色。当在大量数据上预训练并在多个中等规模或小型图像识别基准（ImageNet、CIFAR-100、VTAB 等）上进行迁移学习时，与最先进的卷积网络相比，视觉 Transformer（ViT）取得了优异的成绩，同时训练所需的计算资源大幅减少。

![](../images/vit_2024-10-27-11-58-39.png)

从上图中可以看出来，整个 ViT 的核心思想是，将整个图像划分成网格，每个网络我们称为一个图像 Patch，将这些 Patch 铺平为一维的，那么整个2D 的图像就可以看成是一个由 Patches 组成的一个序列输出。一旦图像转换为了序列向量的形式，那么整个模型的主干就可以使用一个 TransformerBlock 组成的多层 Transformer Encoder 结构。

论文中实现了三种不同大小的模型结构：

|Model|Layers|Hidden size $D$| MLP size | Heads | Params |
|:---:|:---:|:---:|:---:|:---:|:---:|
|ViT-Base|12|768|3072|12|86M|
|ViT-Large|24|1024|4096|16|307M|
|ViT-Huge|32|1280|5120|16|632M|

# 关键模块的从零实现

## PatchEmbedding

ViT 的核心就是把一幅图像看成是一个若干个 16x16 的图像 Patches，每个图像 patch 就看作是 NLP 中的一个 word。所以 ViT 模型的第一步就是把图像划分为 Patch，有两种主流的实现方式：

1. 一种是通过 Reshape + Permute 将图像转换为 Patch，然后经过一个 MLP 进行维度变换。
2. 另一种是直接采用一个卷积的滑动窗口，滑动的步长等于窗口的大小。

在 Transformers 库中，它是经过 transformers.models.vit 中的`ViTEmbeddings` 实现，内部实际是就是通过 `Conv2d`来实现的。

In [1]:
import torch
from torch import nn

batch_size = 1
img_size = 224
patch_size = 16
in_chans = 3
embed_dim = 1024

img = torch.randn(batch_size, in_chans, img_size, img_size)

In [2]:
# 实现方案一：
proj = nn.Conv2d(in_chans, embed_dim, patch_size, patch_size)
x = proj(img)  # batch_size, embed_dim, h_patches, w_patches
x = x.flatten(2).transpose(1, 2)  # NCHW -> NLC
print(x.shape)

torch.Size([1, 196, 1024])


In [3]:
# 实现方案二：
patch_h = img.shape[2] // patch_size
patch_w = img.shape[3] // patch_size
x = img.reshape(batch_size, in_chans, patch_h, patch_size, patch_w, patch_size)
x = x.permute(0, 2, 4, 3, 5, 1).reshape(
    batch_size, -1, patch_size * patch_size * in_chans
)

proj = nn.Linear(patch_size * patch_size * in_chans, embed_dim)
x = proj(x)
print(x.shape)

torch.Size([1, 196, 1024])


## Position Embedding

ViT 中使用的是一维的可训练的绝对位置编码，

In [4]:
num_patches = x.shape[1]
# 考虑上 CLS Token，所以长度为 num_pathces + 1
position_embeddings = nn.Parameter(torch.randn(1, num_patches + 1, embed_dim))

正常情况下，如果输入图像的尺寸正好和预定义的图像尺寸是一致的，那么就直接将 position_embeddings 加到 patch embedding 中去：

In [5]:
cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
cls_tokens = cls_token.expand(x.shape[0], -1, -1)

embeddings = torch.concat([cls_tokens, x], dim=1)  # 添加 CLS Token
embeddings = embeddings + position_embeddings  # 添加位置编码信息

如果输入的图像的尺寸和预定义的尺寸不一致，这时候，我们的位置编码矩阵就需要进行插值。

In [6]:
img = torch.randn(batch_size, in_chans, 384, 384)

proj = nn.Conv2d(in_chans, embed_dim, patch_size, patch_size)
x = proj(img)
x = x.flatten(2).transpose(1, 2)
embeddings = torch.concat([cls_tokens, x], dim=1)  # 添加 CLS Token

height = img.shape[2]
width = img.shape[3]
new_height = height // patch_size
new_width = width // patch_size

cls_pos_embedding = position_embeddings[:, :1, :]
patch_pos_embedding = position_embeddings[:, 1:, :]

patch_pos_embedding = patch_pos_embedding.reshape(1, patch_h, patch_w, embed_dim)
patch_pos_embedding = patch_pos_embedding.permute(0, 3, 1, 2)  # NHWC -> NCHW

patch_pos_embedding = nn.functional.interpolate(
    patch_pos_embedding,
    size=(new_height, new_width),
    mode="bicubic",
    align_corners=False,
)

patch_pos_embedding = patch_pos_embedding.permute(0, 2, 3, 1).reshape(1, -1, embed_dim)
position_embeddings = torch.concat([cls_pos_embedding, patch_pos_embedding], dim=1)

embeddings = embeddings + position_embeddings

# 使用 🤗 Transformers 库

Transformers 中实现了 ViT 架构的模型，类名为 `ViTModel`，我们通过下面构造函数的代码，可以看出，整个 ViTModel 由三个部分组成：

1. ViTEmbeddings: 图像 Patch 化的预处理
2. ViTEncoder: Transformer Encoder
3. ViTPooler: 分类头

```python

class ViTModel(nn.Moduel):
    def __init__(self, config: ViTConfig, add_pooling_layer: bool = True, use_mask_token: bool = False):
        super().__init__(config)
        self.config = config

        self.embeddings = ViTEmbeddings(config, use_mask_token=use_mask_token)
        self.encoder = ViTEncoder(config)

        self.layernorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.pooler = ViTPooler(config) if add_pooling_layer else None

        # Initialize weights and apply final processing
        self.post_init()
```

In [7]:
from timm.models import vision_transformer
from transformers import ViTModel, ViTConfig
import torch

config = ViTConfig()

vit_model = ViTModel(config)

img = torch.randn(1, 3, 384, 384)
vit_output = vit_model(pixel_values=img, interpolate_pos_encoding=True)
print(vit_output.last_hidden_state.shape)
print(vit_output.pooler_output.shape)

torch.Size([1, 577, 768])
torch.Size([1, 768])
